# Getting started with pyglotaran

What is pyglotaran?

What is this notebook about?
- You have some data you want to analyze. Probably .csv. We're going to show you how to import that later. First, let's go through the workflow.

You have some data
You want to plot your data, check that it's ok. 
Some notes about checking if data is ok. 
- No NaNs, no Ing.
- No duplicate x or y values. Monotonic increasing.

Once data ok, we want to fit the data. What do we need?
We need a model, with some parameters and starting values for those parameters.
We will organize this within a project.

Let us get some example data to analyze:

In [ ]:
from glotaran.testing.simulated_data.sequential_spectral_decay import DATASET as my_dataset

my_dataset

Like all data in `pyglotaran`, the dataset is a [xarray.Dataset](https://xarray.pydata.org/en/stable/api.html#dataset).
You can find more information about the ``xarray`` library the [xarray hompage](https://xarray.pydata.org/en/stable/).

The loaded dataset is a simulated sequential model.

## Plotting raw data
Now lets plot some time traces.

In [ ]:
from pyglotaran_extras.plotting import plot_data_overview

plot_data_overview(my_dataset);

We can also plot spectra at different times.

## Starting a project

To start using `pyglotaran` in your analysis, you only have to import the `Project` class 
and open a project.

In [ ]:
from glotaran.project import Project

my_project = Project.open("my_project")
my_project

If the project does not already exist this will create a new project and its folder structure for 
you. In our case we had only the `models` + `parameters` folders and the `data` + `results` folder
were created when opening the project.

In [ ]:
%ls my_project

# Import the data into your project

As long as you can read your data into a `xarray.Dataset` or `xarray.DataArray` you can directly 
import it in to your project. 

This will save your data as `NetCDF` (`.nc`) file into the data 
folder inside of your project with the name that you gave it (here `my_project/data/my_data.nc`).

If the data format you are using is supported by a plugin you can simply copy the file to
the data folder of the project (here `my_project/data`).

In [ ]:
my_project.import_data(my_dataset, dataset_name="my_data")
my_project

After importing our `my_project` is aware of the data that we named `my_data` when importing.

## Preparing data
To get an idea about how to model your data, you should inspect the singular
value decomposition. 
As a convenience the `load_data` method has the option to add svd data on the fly.

In [ ]:
dataset_with_svd = my_project.load_data("my_data", add_svd=True)
dataset_with_svd

First, take a look at the first 10 singular values:

In [ ]:
plot_data = dataset_with_svd.data_singular_values.sel(singular_value_index=range(10))
plot_data.plot(yscale="log", marker="o", linewidth=0, aspect=2, size=5);

This tells us that our data have at least three components which we need to model.

## Working with models

To analyze our data, we need to create a model. 

Create a file called `my_model.yaml` in your projects `models` directory and fill it with the 
following content.

In [ ]:
my_project.show_model_definition("my_model")

You can check your model for problems with the `validate` method.

In [ ]:
my_project.validate("my_model")

## Working with parameters

Now define some starting parameters. Create a file called `parameters.yaml` in your projects 
`parameters` directory with the following content.

In [ ]:
my_project.show_parameters_definition("my_parameters")

Note the `{ "vary": False }` which tells `pyglotaran` that those parameters should not be changed.

You can use `validate` method also to check for missing parameters.

In [ ]:
my_project.validate("my_model", "my_parameters")

Since not all problems in the model can be detected automatically it is wise to
visually inspect the model. For this purpose, you can just load the model and inspect its markdown 
rendered version.

In [ ]:
my_project.load_model("my_model")

The same way you should inspect your parameters.

In [ ]:
my_project.load_parameters("my_parameters")

## Optimizing data

Now we have everything together to optimize our parameters.

In [ ]:
result = my_project.optimize("my_model", "my_parameters")
result

Each time you run an optimization the result will be saved in the projects results folder.

In [ ]:
%ls "quickstart_project/results"

To visualize how quickly the optimization converged we ca plot the optimality of the 
`optimization_history`.

In [ ]:
result.optimization_history.data["optimality"].plot(logy=True)

In [ ]:
result.optimized_parameters

You can inspect the data of your `result` by accessing `data` attribute. In our example it only 
contains our single `my_data` dataset, but it ca contain as many dataset as you analysis needs.

In [ ]:
result.data

## Visualize the Result

The resulting data can be visualized the same way as the dataset. To judge the
quality of the fit, you should look at first left and right singular vectors of
the residual.

In [ ]:
result_dataset = result.data["my_data"]

residual_left = result_dataset.residual_left_singular_vectors.sel(left_singular_value_index=0)
residual_right = result_dataset.residual_right_singular_vectors.sel(right_singular_value_index=0)
residual_left.plot.line(x="time", aspect=2, size=5)
residual_right.plot.line(x="spectral", aspect=2, size=5);